In [1]:
import re
import time
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from kerastuner.tuners import RandomSearch
from tensorflow.keras.optimizers import Adam
from kerastuner.tuners import RandomSearch

C:\Users\Bureau\AppData\Local\Temp\ipykernel_3768\650880806.py:15: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


# Chargement des données et préprocessing

In [2]:
# Chargement des données
df=pd.read_csv('X_train_update.csv')
df1=pd.read_csv("y_train_CVw08PX.csv")

In [3]:
# Préprosessing des données
## Codage de la colone prdttype de 1 à 27
l=[]
for mm in df1.prdtypecode:
    if mm not in l :
        l.append(mm)
l.sort()
for i, j in enumerate(l) :
    for mmm in df1.prdtypecode :
        if mmm==j:
            df1[df1.prdtypecode==mmm]=i+1

## Regroupement de la variable description et designation             
df["description"] = (df["designation"].fillna('') + " " + df["description"].fillna('')).str.lower()
df["categ"] = df1.prdtypecode

##Nétoyage
stop_words = set([
    "le", "la", "les", "de", "du", "des", "et", "en", "un", "une", "pour", 
    "avec", "dans", "par", "sur", "au", "aux", "ce", "ces", "se", "ses", 
    "est", "qui", "que", "dont", "ou", "à", "son", "sa", "leurs", "comme"
])
stop_words_fr = set(stopwords.words("french"))
stop_words_en = set(stopwords.words("english"))
stop_words_de = set(stopwords.words("german"))
stop_words_es = set(stopwords.words("spanish"))
stop_words_it = set(stopwords.words("italian"))
stop_words_pt = set(stopwords.words("portuguese"))
stop_words_misc = set(["br", "li", "x", "p", "ul", "div", "strong", "nbsp", "eacute", "amp", ])  # Mots divers à ignorer

# Fusionner les stopwords
stop_words_all = stop_words_fr | stop_words_en | stop_words_de | stop_words_es | stop_words_it | stop_words_pt | stop_words_misc | stop_words

##
def nettoyer_texte(texte):
    texte = str(texte).lower()  # On met en minuscule
    texte = re.sub(r'\d+', '', texte)  # On supprine les chiffres
    texte = re.sub(r'\W+', ' ', texte)  # On supprime la ponctuation et caractères spéciaux
    texte = ' '.join([mot for mot in texte.split() if mot not in stop_words_all])  # On supprime les stopwords personnalisés
    return texte
##     
df['description'] = df['description'].apply(nettoyer_texte)

# Modèle simple

In [4]:
# Stokons dans X les variables explicatives et dans y la variable cible.
X=df.description
y=df.categ

In [5]:
#  Paramètres
max_vocab_size = 20000  # Vocabulaire maximum
max_sequence_length = 200  # Longueur des séquences
embedding_dim = 100  # Dimension de l'embedding

#  Tokenisation
tokenizer = Tokenizer(num_words=max_vocab_size)
tokenizer.fit_on_texts(X)
sequences = tokenizer.texts_to_sequences(X)

X1 = pad_sequences(sequences, maxlen=max_sequence_length)

X1 = np.array(X1)
y1 = np.array(y)

#  Séparons les données en un ensemble d'apprentissage et un ensemble de test (20%)
X_train, X_test, y_train, y_test = train_test_split(X1, y1, test_size=0.2, random_state=42)


In [6]:

# Modèle CNN
model = Sequential([
    Embedding(input_dim=max_vocab_size, output_dim=embedding_dim, input_length=max_sequence_length),
    Conv1D(128, 5, activation='relu'),
    GlobalMaxPooling1D(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Binaire
])



# Compilation
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Entraînement
model.fit(X_train, y_train, epochs=20, batch_size=128, validation_split=0.1)


Epoch 1/20


C:\Users\Bureau\Desktop\projetRakuten\venv\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


478/478 ━━━━━━━━━━━━━━━━━━━━ 22s 43ms/step - accuracy: 0.0385 - loss: -73846960.0000 - val_accuracy: 0.0365 - val_loss: -2245763072.0000
Epoch 2/20
478/478 ━━━━━━━━━━━━━━━━━━━━ 20s 42ms/step - accuracy: 0.0361 - loss: -6993678848.0000 - val_accuracy: 0.0365 - val_loss: -38484566016.0000
Epoch 3/20
478/478 ━━━━━━━━━━━━━━━━━━━━ 20s 41ms/step - accuracy: 0.0379 - loss: -63292166144.0000 - val_accuracy: 0.0365 - val_loss: -178506006528.0000
Epoch 4/20
478/478 ━━━━━━━━━━━━━━━━━━━━ 21s 43ms/step - accuracy: 0.0376 - loss: -242951471104.0000 - val_accuracy: 0.0365 - val_loss: -502976806912.0000
Epoch 5/20
478/478 ━━━━━━━━━━━━━━━━━━━━ 20s 43ms/step - accuracy: 0.0367 - loss: -624233611264.0000 - val_accuracy: 0.0365 - val_loss: -1089533509632.0000
Epoch 6/20
478/478 ━━━━━━━━━━━━━━━━━━━━ 20s 42ms/step - accuracy: 0.0381 - loss: -1282402549760.0000 - val_accuracy: 0.0365 - val_loss: -2015576719360.0000
Epoch 7/20
478/478 ━━━━━━━━━━━━━━━━━━━━ 20s 42ms/step - accuracy: 0.0379 - loss: -230708346880

In [7]:
# Évaluation
y_pred=model.predict(X_test)
loss, acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {acc:.4f}")

# Calcul du F1-score moyen (macro, micro ou weighted selon ton besoin)
for f1_sc in ["macro", "micro", "weighted"]:
    f1 = f1_score(y_test, y_pred, average=f1_sc)  
    print(f"F1-score moyen {f1_sc}:", f1)

531/531 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step
531/531 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.0363 - loss: -91791067971584.0000
Test Accuracy: 0.0360
F1-score moyen macro: 0.0025763431082821853
F1-score moyen micro: 0.03603391427225624
F1-score moyen weighted: 0.002506564620893478


Les performances sont de très mauvaises, nous allons approfondire par une cross-validation. 

# Cross-validation

In [8]:

#  Données
X=df.description.astype(str).tolist()
#y=df.categ.values
num_classes = len(np.unique(y))

#  Encodage des labels


label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y=df.categ)  # valeurs entre 0 et num_classes-1
y_encoded = to_categorical(y)  # one-hot encoding

#  Paramètres
max_vocab_size = 20000
max_sequence_length = 200
embedding_dim = 100
num_folds = 5

# Tokenisation
tokenizer = Tokenizer(num_words=max_vocab_size)
tokenizer.fit_on_texts(X)
sequences = tokenizer.texts_to_sequences(X)
Xcnn = pad_sequences(sequences, maxlen=max_sequence_length)

#  Cross-validation
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)
fold = 1
acc_per_fold = []

f1_per_fold = []

for train_index, val_index in kf.split(Xcnn):
    print(f"\n Fold {fold}")
    X_train, X_val = Xcnn[train_index], Xcnn[val_index]
    y_train, y_val = y_encoded[train_index], y_encoded[val_index]

    #  CNN modèle
    model = Sequential([
        Embedding(input_dim=max_vocab_size, output_dim=embedding_dim, input_length=max_sequence_length),
        Conv1D(128, 5, activation='relu'),
        GlobalMaxPooling1D(),
        Dense(64, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    #  Entraînement
    model.fit(X_train, y_train, epochs=20, batch_size=128, verbose=1)

    #  Évaluation
    scores = model.evaluate(X_val, y_val, verbose=0)
    print(f" Fold {fold} accuracy: {scores[1]:.4f}")
    acc_per_fold.append(scores[1])

    # Prédictions
    y_val_pred_probs = model.predict(X_val)
    y_val_pred = np.argmax(y_val_pred_probs, axis=1)
    y_val_true = np.argmax(y_val, axis=1)

    # F1-score weighted
    f1 = f1_score(y_val_true, y_val_pred, average='weighted')
    print(f" Fold {fold} F1-score pondéré: {f1:.4f}")
    f1_per_fold.append(f1)

    fold += 1

#  Résumé
print("\n Résultats par fold:")
for i, (acc, f1) in enumerate(zip(acc_per_fold, f1_per_fold)):
    print(f"Fold {i+1}: Accuracy = {acc:.4f}, F1-score pondéré = {f1:.4f}")

print(f"\n Moyenne de l’accuracy : {np.mean(acc_per_fold):.4f}")
print(f" Moyenne du F1-score pondéré : {np.mean(f1_per_fold):.4f}")


 Fold 1
Epoch 1/20


C:\Users\Bureau\Desktop\projetRakuten\venv\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


531/531 ━━━━━━━━━━━━━━━━━━━━ 22s 40ms/step - accuracy: 0.3212 - loss: 2.3837
Epoch 2/20
531/531 ━━━━━━━━━━━━━━━━━━━━ 22s 41ms/step - accuracy: 0.7516 - loss: 0.8597
Epoch 3/20
531/531 ━━━━━━━━━━━━━━━━━━━━ 21s 40ms/step - accuracy: 0.8305 - loss: 0.5877
Epoch 4/20
531/531 ━━━━━━━━━━━━━━━━━━━━ 21s 40ms/step - accuracy: 0.8701 - loss: 0.4515
Epoch 5/20
531/531 ━━━━━━━━━━━━━━━━━━━━ 22s 40ms/step - accuracy: 0.8983 - loss: 0.3474
Epoch 6/20
531/531 ━━━━━━━━━━━━━━━━━━━━ 21s 40ms/step - accuracy: 0.9192 - loss: 0.2730
Epoch 7/20
531/531 ━━━━━━━━━━━━━━━━━━━━ 21s 40ms/step - accuracy: 0.9353 - loss: 0.2171
Epoch 8/20
531/531 ━━━━━━━━━━━━━━━━━━━━ 21s 40ms/step - accuracy: 0.9467 - loss: 0.1785
Epoch 9/20
531/531 ━━━━━━━━━━━━━━━━━━━━ 21s 40ms/step - accuracy: 0.9541 - loss: 0.1526
Epoch 10/20
531/531 ━━━━━━━━━━━━━━━━━━━━ 21s 40ms/step - accuracy: 0.9599 - loss: 0.1315
Epoch 11/20
531/531 ━━━━━━━━━━━━━━━━━━━━ 22s 41ms/step - accuracy: 0.9642 - loss: 0.1201
Epoch 12/20
531/531 ━━━━━━━━━━━━━━━━━━━━ 

C:\Users\Bureau\Desktop\projetRakuten\venv\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


531/531 ━━━━━━━━━━━━━━━━━━━━ 22s 40ms/step - accuracy: 0.3216 - loss: 2.3732
Epoch 2/20
531/531 ━━━━━━━━━━━━━━━━━━━━ 21s 40ms/step - accuracy: 0.7517 - loss: 0.8671
Epoch 3/20
531/531 ━━━━━━━━━━━━━━━━━━━━ 21s 40ms/step - accuracy: 0.8247 - loss: 0.5997
Epoch 4/20
531/531 ━━━━━━━━━━━━━━━━━━━━ 21s 40ms/step - accuracy: 0.8678 - loss: 0.4506
Epoch 5/20
531/531 ━━━━━━━━━━━━━━━━━━━━ 21s 40ms/step - accuracy: 0.8949 - loss: 0.3494
Epoch 6/20
531/531 ━━━━━━━━━━━━━━━━━━━━ 21s 40ms/step - accuracy: 0.9159 - loss: 0.2796
Epoch 7/20
531/531 ━━━━━━━━━━━━━━━━━━━━ 21s 40ms/step - accuracy: 0.9325 - loss: 0.2279
Epoch 8/20
531/531 ━━━━━━━━━━━━━━━━━━━━ 22s 40ms/step - accuracy: 0.9445 - loss: 0.1869
Epoch 9/20
531/531 ━━━━━━━━━━━━━━━━━━━━ 22s 41ms/step - accuracy: 0.9503 - loss: 0.1632
Epoch 10/20
531/531 ━━━━━━━━━━━━━━━━━━━━ 22s 42ms/step - accuracy: 0.9579 - loss: 0.1432
Epoch 11/20
531/531 ━━━━━━━━━━━━━━━━━━━━ 22s 41ms/step - accuracy: 0.9632 - loss: 0.1245
Epoch 12/20
531/531 ━━━━━━━━━━━━━━━━━━━━ 

...

531/531 ━━━━━━━━━━━━━━━━━━━━ 23s 41ms/step - accuracy: 0.3188 - loss: 2.3788
Epoch 2/20
531/531 ━━━━━━━━━━━━━━━━━━━━ 22s 41ms/step - accuracy: 0.7473 - loss: 0.8790
Epoch 3/20
531/531 ━━━━━━━━━━━━━━━━━━━━ 22s 41ms/step - accuracy: 0.8271 - loss: 0.5973
Epoch 4/20
531/531 ━━━━━━━━━━━━━━━━━━━━ 22s 41ms/step - accuracy: 0.8726 - loss: 0.4464
Epoch 5/20
531/531 ━━━━━━━━━━━━━━━━━━━━ 22s 41ms/step - accuracy: 0.8985 - loss: 0.3465
Epoch 6/20
531/531 ━━━━━━━━━━━━━━━━━━━━ 22s 41ms/step - accuracy: 0.9219 - loss: 0.2634
Epoch 7/20
531/531 ━━━━━━━━━━━━━━━━━━━━ 22s 41ms/step - accuracy: 0.9359 - loss: 0.2187
Epoch 8/20
531/531 ━━━━━━━━━━━━━━━━━━━━ 22s 41ms/step - accuracy: 0.9479 - loss: 0.1782
Epoch 9/20
531/531 ━━━━━━━━━━━━━━━━━━━━ 22s 41ms/step - accuracy: 0.9546 - loss: 0.1528
Epoch 10/20
531/531 ━━━━━━━━━━━━━━━━━━━━ 22s 41ms/step - accuracy: 0.9597 - loss: 0.1330
Epoch 11/20
531/531 ━━━━━━━━━━━━━━━━━━━━ 22s 41ms/step - accuracy: 0.9649 - loss: 0.1175
Epoch 12/20
531/531 ━━━━━━━━━━━━━━━━━━━━ 

C:\Users\Bureau\Desktop\projetRakuten\venv\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


531/531 ━━━━━━━━━━━━━━━━━━━━ 23s 42ms/step - accuracy: 0.3175 - loss: 2.4058
Epoch 2/20
531/531 ━━━━━━━━━━━━━━━━━━━━ 22s 41ms/step - accuracy: 0.7467 - loss: 0.8712
Epoch 3/20
531/531 ━━━━━━━━━━━━━━━━━━━━ 22s 41ms/step - accuracy: 0.8265 - loss: 0.6028
Epoch 4/20
531/531 ━━━━━━━━━━━━━━━━━━━━ 22s 41ms/step - accuracy: 0.8636 - loss: 0.4517
Epoch 5/20
531/531 ━━━━━━━━━━━━━━━━━━━━ 22s 41ms/step - accuracy: 0.8979 - loss: 0.3466
Epoch 6/20
531/531 ━━━━━━━━━━━━━━━━━━━━ 22s 41ms/step - accuracy: 0.9163 - loss: 0.2832
Epoch 7/20
531/531 ━━━━━━━━━━━━━━━━━━━━ 22s 41ms/step - accuracy: 0.9324 - loss: 0.2238
Epoch 8/20
531/531 ━━━━━━━━━━━━━━━━━━━━ 22s 41ms/step - accuracy: 0.9448 - loss: 0.1865
Epoch 9/20
531/531 ━━━━━━━━━━━━━━━━━━━━ 22s 41ms/step - accuracy: 0.9488 - loss: 0.1657
Epoch 10/20
531/531 ━━━━━━━━━━━━━━━━━━━━ 22s 42ms/step - accuracy: 0.9569 - loss: 0.1407
Epoch 11/20
531/531 ━━━━━━━━━━━━━━━━━━━━ 22s 41ms/step - accuracy: 0.9620 - loss: 0.1238
Epoch 12/20
531/531 ━━━━━━━━━━━━━━━━━━━━ 

C:\Users\Bureau\Desktop\projetRakuten\venv\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


531/531 ━━━━━━━━━━━━━━━━━━━━ 23s 41ms/step - accuracy: 0.3193 - loss: 2.4045
Epoch 2/20
531/531 ━━━━━━━━━━━━━━━━━━━━ 22s 41ms/step - accuracy: 0.7508 - loss: 0.8656
Epoch 3/20
531/531 ━━━━━━━━━━━━━━━━━━━━ 22s 41ms/step - accuracy: 0.8315 - loss: 0.5796
Epoch 4/20
531/531 ━━━━━━━━━━━━━━━━━━━━ 22s 41ms/step - accuracy: 0.8699 - loss: 0.4453
Epoch 5/20
531/531 ━━━━━━━━━━━━━━━━━━━━ 22s 41ms/step - accuracy: 0.9018 - loss: 0.3404
Epoch 6/20
531/531 ━━━━━━━━━━━━━━━━━━━━ 22s 42ms/step - accuracy: 0.9195 - loss: 0.2716
Epoch 7/20
531/531 ━━━━━━━━━━━━━━━━━━━━ 22s 41ms/step - accuracy: 0.9359 - loss: 0.2161
Epoch 8/20
531/531 ━━━━━━━━━━━━━━━━━━━━ 22s 41ms/step - accuracy: 0.9466 - loss: 0.1804
Epoch 9/20
531/531 ━━━━━━━━━━━━━━━━━━━━ 22s 41ms/step - accuracy: 0.9552 - loss: 0.1535
Epoch 10/20
531/531 ━━━━━━━━━━━━━━━━━━━━ 22s 41ms/step - accuracy: 0.9598 - loss: 0.1354
Epoch 11/20
531/531 ━━━━━━━━━━━━━━━━━━━━ 22s 41ms/step - accuracy: 0.9641 - loss: 0.1178
Epoch 12/20
531/531 ━━━━━━━━━━━━━━━━━━━━ 